# Urbanization comparison

Compare the multiple urban extents in the project

1. Official administrative boundaries provided by the project team.  
  a. Need to get metadata on these boundaries from Swati  
2. Official administrative boundaries with buffers  
  a. Why these specific buffer distances?  
3. DoU boundaries  
  a. Which population source  


In [28]:
import sys, os
import rasterio

import pandas as pd
import geopandas as gpd

import GOSTurban.UrbanRaster as urban


In [24]:
base_folder = "C:/WBG/Work/LKA_URB_CCDR/Data"
urban_folder = os.path.join(base_folder, "Urban_extents")
population_folder = os.path.join(base_folder, "Population")

worldpop_file = os.path.join(population_folder, "lka_ppp_2020_UNadj.tif")
combo_extents = os.path.join(urban_folder, "combo_extents.shp")

In [18]:
# Combine all the urban extents into a single file
if not os.path.exists(combo_extents):
    urban_extents = [x for x in os.listdir(os.path.join(urban_folder, "Final_AoI")) if x.endswith(".shp")]
    all_extents = []
    for urban_extent in urban_extents:
        urban_extent_gdf = gpd.read_file(os.path.join(urban_folder, "Final_AoI", urban_extent))
        type = urban_extent.split("_")[1].replace(".shp", "").lower()
        if type == "colombo":
            type = "city"
            if "buffer" in urban_extent:
                type = "20km"
        urban_extent_gdf["Type"] = type
        all_extents.append(urban_extent_gdf)

    all_extents_gdf = gpd.GeoDataFrame(pd.concat(all_extents, ignore_index=True))
    all_extents_gdf.to_file(combo_extents)
else:
    all_extents_gdf = gpd.read_file(combo_extents)

In [29]:
# extract urban extents from population grids
for pop_layer, name in [[worldpop_file, "WorldPop"]]:
    urb_out_file = os.path.join(urban_folder, "DOU_urb_extent_%s.shp" % name)
    hd_out_file = os.path.join(urban_folder, "DOU_hd_extent_%s.shp" % name)
    if not os.path.exists(hd_out_file):
        curPop = rasterio.open(pop_layer)
        urban_calculator = urban.urbanGriddedPop(curPop)
        urban_extents = urban_calculator.calculateUrban(
            densVal=3, totalPopThresh=5000, smooth=False, queen=False, verbose=True
        )
        urban_extents['Type'] = name
        urban_extents.to_file(urb_out_file)

        hd_urban_extents = urban_calculator.calculateUrban(
            densVal=15,
            totalPopThresh=50000,
            smooth=True,
            queen=True,  # high density extents use queen's case contiguity, and
            verbose=True,
        )  # High density extents have hole smoothing applied.
        hd_urban_extents['Type'] = name
        hd_urban_extents.to_file(hd_out_file)


    


13:19:42	: Read in urban data
13:19:42	: Creating Shape 0
13:19:45	: Read in urban data
13:19:45	: Creating Shape 0


c:\wbg\Anaconda3\envs\urban_test\Lib\site-packages\GOSTurban\UrbanRaster.py:404: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  xxGeom["geometry"] = xxGeom.buffer((popRaster.res[0] / 2))


ValueError: graph should have two dimensions

In [26]:
dir(GOSTurban)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_version']